In [2]:
import pandas as pd
import os
import json
import csv
import numpy as np
import statistics
import math


In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [4]:
# ielasām 18885 atrastos kermena vārdus un kontekstu no csv faila
df = pd.read_csv('_40_kermena_vardi_vardskira_konteksts_18855_atrasti.csv', index_col=[0])

In [5]:
# skaits ar atrastiem kermena vārdiem
df.shape

(18855, 6)

In [6]:
# noglabājam Random sample 5% visu rindu uz Excel failu, lai ar tuvlasījumu pārbaudītu ķermeņa vārdu īpatsvaru. 


five_percent_sample = df.copy()
five_percent_sample = five_percent_sample.sort_index() 
# five_percent_sample.to_excel('_81_full_18885_words_sorted.xlsx', encoding='utf8')

# Ņemam katru divdesmito teikumu no visiem atrastajiem teikumiem, kuros ir ķermeņu vārdi.
# Tas nodrošina nejaušu, reprezentatīvu izlasi, kurā ir pārstāvēti visi teksti. 
# Turklāt iespējams novērot vai ir kādas lielākas variācijas noteiktos laika periodos.
five_percent_sample = five_percent_sample.drop(five_percent_sample[five_percent_sample.index % 20 != 0].index)

five_percent_sample = five_percent_sample.sort_index() 
five_percent_sample.head(100)

five_percent_sample.to_excel('_82_five_percent_sample.xlsx', encoding='utf8')

In [7]:
# sakārtojam visus datus tā, lai primāri šķiro pēc VĀRDŠĶIRAS, un sekundāri šķiro pēc LEMMAS – tas dod 
# sākumā visus VERB, pēc tam NOUN, pēc tam ADJ, ADV utml —– pēc tam alfabētiski ––
df = df.sort_values(['vardskira', 'lemma'], ascending=[False, False])

In [8]:
# kā izskatās apvienotais Dataframe – pirmās piecas rindas
df.head(10)

,fleksiva_forma,lemma,vardskira,indekss_teksta,teksta_fragments,faila_nosaukums
21,stāv,stāvēt,VERB,3343,lielākai daļai paša cilvēka varā stāv savu dzīvi darīt priecīgu un,1893MedinskaMarijaSērdienīteStasts
33,stāv,stāvēt,VERB,5812,"pie tādām darīšanām , kuras stāv manā vaļā . Citādi tas",1893MedinskaMarijaSērdienīteStasts
61,stāvam,stāvēt,VERB,2657,", vai mēs pret to stāvam vai ejam bojā ... Laimīgākās",1894AspazijaCinaParNakamibuNovele
127,stāv,stāvēt,VERB,8010,"sadzīves pārijs , izstumtais , stāv viņas troņa priekšā . Visi",1894AspazijaCinaParNakamibuNovele
146,stāvu,stāvēt,VERB,9760,", tagad cieši apkampa viņas stāvu . Viņai pie šī apskāviena",1894AspazijaCinaParNakamibuNovele
158,stāv,stāvēt,VERB,11916,", savu mērķi sasniegusi , stāv klusu vai nu iet atpakaļ",1894AspazijaCinaParNakamibuNovele
169,stāv,stāvēt,VERB,13153,"gan tie muļķi , kuri stāv , rokas salikuši , un",1894AspazijaCinaParNakamibuNovele
208,stāvam,stāvēt,VERB,16398,"it kā sastingusi uz vietas stāvam no kāda nāvīgi auksta ,",1894AspazijaCinaParNakamibuNovele
232,stāv,stāvēt,VERB,1049,", kā tava vīra veikals stāv , » viņš atteica jau",1895ZaliteHermineEjUnIzglitojiesStasts
237,stāv,stāvēt,VERB,1914,"ziniet , ka manā varā stāv , ka rītu šis dzīvoklis",1895ZaliteHermineEjUnIzglitojiesStasts


# Komentāri ejot cauri failam un skatoties pirmkārt izņēmumus



## 3 veidu ķermeņa vārdi – (1) īstie, (2) ķermeņa vārdi lietoti kā metaforas, (3) ķermeņa vārdi iekš idiomātiskām frāzēm.


### (2) ķermeņa vārdi lietoti kā metaforas

15194 – mīkstu bērna roku kā gliemenes pēdiņu pāri galdam pēc saldskābmaizes šķēles

13858 – ezers gluds un plats kā plauksta , un nekas , gluži

6605 – zari kā brūni , mezglaini pirksti sliecas laukā no dziļā sniega

1155 – gausi līda pār melnā kalna muguru uz augšu . Olīvu galotnes

14012 – tante aizdomīgi nopētīja krēslu tievās kājiņas , kāds onkulis pārsteigts uzlūkoja



### (3) ķermeņa vārdi iekš idiomātiskām frāzēm

– neitrāls ķermeņa darbību apraksts:

1640 – sāns – sievietes mani arvienu uzlūkotu no sāniem – bijušu tingeltangeļa dziedātāju ;

18130 – ... tā , tā - rokā ir - Kaijgarde Hildegarde ,

421 – domā , ka sirdsapziņa bez pēdām pazuduse ! Bet šādas domas

741 – pārmeta māsai . Irena paraustīja plecus . « Nezinu , kāds	

414 – balsī , tumši raudzīdamies caur pieri . Vizbulis arī rādījās uzbudināts

8016 – visi brītiņu paliek kā uz mutes sisti . — « Labdien	

14404 – padomājiet pati , viens kā pirksts tai bēdu ielejā , kulies

1212 – pareizāki griezt nule piedzīvotam atgadījumam muguru un ar ārstu uz priekšu

1812 – . Viņa drebēja pie visas miesas un vērās bezprātīgiem sāpju mirkļiem	

4785 – kādu sulu viņa barojusies mātes miesās ? " " Ar labāko	

1816 – un es jutu ka man mati slienas stāvu ... Tur zemē

1465 – draugs ! Viņa spītīgi saknieba lūpas . Nē , nē ,

98 – atrodas kādi no cilvēces izstumti locekļi , bajadēras , kuras mājo	

200 – pienāca , visa pilsēta bija kājās . Dāmas aizmirsa savus vājos	

303 – līdzi nabadzītes , Kurām sāpes krūtis spiež , Kuras viltus mīlestības

8 – , kas še tagad zemes klēpī dus , – tas bij

530 –  Sergejs smejas nicinoši pilnā kaklā , stāvēdams atvērtajās koridora durvīs

4221 – kuras varēja manīt vīna pudeles kaklu . „ Lai slavēts Kungs

6907 – Nevaru , — Nija papurināja galvu un novērsās . Viņu smacēja

7482 – teica tas viens runātājs no galvas pilsētas ? ... Kapsēta ir

38 – tagad staltāks – pilnīgā vīra augumā . Vaj patiesi nezināji ,



## kļūdas un nepareizības – morfoloģiskais pārseris, homonīmi, transkribēšanas

### nepareizi atpazītas part-of-speech

146 – atpazīts kā "VERB" "stāv" – , tagad cieši apkampa viņas stāvu . Viņai pie šī apskāviena	

6505 – atpazīts kā kā "VERB" rakt – pūra lādes vāks . lešāvusi roku pašā dibenā , viņa brīdi



### morfoloģiskā pārsera kļūdas pie lemmatizēšanas

2591 – atpazīts kā "VERB" mutīt – un paļāvīgi ver savu sakarsušo mutīti manai tīrīšanai . Kad es	

6541 – atpazīts kā PROPN Matis – , tev ir balti linu mati , zilas linu ziedu acis

856 uc – fleksīvā forma "seju" ir pārvērsta uz lemmu "sejs", atpazīta kā "NOUN"

1215, 7734, 18023 – "roka" lemmatizēts kā "roks" NOUN

5 gadījumos fleksīvā forma "mutīti" ir pārvērsta uz lemmu "mutītis", atpazīta kā "NOUN"

5 gadījumos fleksīvā forma "deguntiņu" ir pārvērsta uz lemmu "deguntiņa", atpazīta kā "NOUN"



### homonīmi, kas nav ķermeņa vārdi

2991 – palodzes . — No piektā stāva līdz apakšai — tāļi ,

481 – kas saujā . Viņš to cirta pret grīdu un sauca : ––– atpazīts kā "cirta" "NOUN"



### transkribēšanas kļūdas ievadot tekstus? 

2394 – šķiet, nepareizi transkribēts "galva"/ "galda" – . Ātri viņa piecēlās no galva un karsts sārtums saplūda tās

izskatās, ka dažās vietās trūkst diakritisko zīmju "celi" / "ceļi":
    
15545 – pļavas , ganības un putekļaini celi . Nekas nebango , nemutuļo

deguntiņs – nepareizi transkribēts oriģinālfailā visdrīzāk




## Papildus komentāri
    
ceļš – ļoti daudz, jāskata manuāli cauri, jo tur iekšā ir arī "celis" daudzkārt

celis / ceļš – daudzi atpazīti kā "ceļš":

19 – ka varētu iet patstāvīgi dzīves ceļu . Un neaizmirsti arī ,

"stāvi" – daudzi nepareizi atpazīti – kā "ADV":

3731 – ilgoto atbraucēju . Kad abi stāvi , viens otram piekļāvušies ,

stāvēt VERB – varētu būt pārsvarā pareizi atpazīti darbības vārdi, taču varētu būt iekšā arī ķermeņa daļas

stāvs ADJ, stāvus, stāvi ADV – varētu būt iekšā arī kādi ķermeņa "stāvi"

? – rakt VERB, galva PROPN, galviņu PROPN, krūšu NOUN, Matis PROPN, deguntiņa –

seja – ļoti daudz

roka – ļoti daudz


In [9]:
df_nouns_only = df.copy()

In [10]:
df_non_nouns = df.copy()
df_non_nouns = df_non_nouns.loc[df_non_nouns['vardskira'] != "NOUN"]
df_non_nouns


,fleksiva_forma,lemma,vardskira,indekss_teksta,teksta_fragments,faila_nosaukums
21,stāv,stāvēt,VERB,3343,lielākai daļai paša cilvēka varā stāv savu dzīvi darīt priecīgu un,1893MedinskaMarijaSērdienīteStasts
33,stāv,stāvēt,VERB,5812,"pie tādām darīšanām , kuras stāv manā vaļā . Citādi tas",1893MedinskaMarijaSērdienīteStasts
61,stāvam,stāvēt,VERB,2657,", vai mēs pret to stāvam vai ejam bojā ... Laimīgākās",1894AspazijaCinaParNakamibuNovele
127,stāv,stāvēt,VERB,8010,"sadzīves pārijs , izstumtais , stāv viņas troņa priekšā . Visi",1894AspazijaCinaParNakamibuNovele
146,stāvu,stāvēt,VERB,9760,", tagad cieši apkampa viņas stāvu . Viņai pie šī apskāviena",1894AspazijaCinaParNakamibuNovele
158,stāv,stāvēt,VERB,11916,", savu mērķi sasniegusi , stāv klusu vai nu iet atpakaļ",1894AspazijaCinaParNakamibuNovele
169,stāv,stāvēt,VERB,13153,"gan tie muļķi , kuri stāv , rokas salikuši , un",1894AspazijaCinaParNakamibuNovele
208,stāvam,stāvēt,VERB,16398,"it kā sastingusi uz vietas stāvam no kāda nāvīgi auksta ,",1894AspazijaCinaParNakamibuNovele
232,stāv,stāvēt,VERB,1049,", kā tava vīra veikals stāv , » viņš atteica jau",1895ZaliteHermineEjUnIzglitojiesStasts
237,stāv,stāvēt,VERB,1914,"ziniet , ka manā varā stāv , ka rītu šis dzīvoklis",1895ZaliteHermineEjUnIzglitojiesStasts


In [11]:
'''
    manuāli caurskatīti NON-NOUNS
    atzīmēti indeksi, kur atrastais vārds tomēr ir NOUN
'''

## TODO – uztaisīt arī COUNT - cik ir no katra, un tā dabūsim procentu ar nepareizi atpazītajiem

temp_list = []

# headers = lemma, vardskira, TOTAL, MISSED_NOUNS

# stāvs ADJ
stavs_WRONG_list = [
    1724,
    7568,
    7751,
    12533,
    14993,
    15203,
    15607,
    15617,
    15713,
    15762
]
print( "stāvs", "ADJ", df_non_nouns[(df_non_nouns['lemma'] == "stāvs")].shape[0], 10 )
temp_list.append(["stāvs", "ADJ", df_non_nouns[(df_non_nouns['lemma'] == "stāvs")].shape[0], 10])

# stāvi ADV
3731
print( "stāvi", "ADV", df_non_nouns[(df_non_nouns['lemma'] == "stāvi")].shape[0], 1 )
temp_list.append(["stāvi", "ADV", df_non_nouns[(df_non_nouns['lemma'] == "stāvi")].shape[0], 1])

# stāvus ADV
3801
3896
5574
15154
15808
print( "stāvus", "ADV", df_non_nouns[(df_non_nouns['lemma'] == "stāvus")].shape[0], 5 ) 
temp_list.append(["stāvus", "ADV", df_non_nouns[(df_non_nouns['lemma'] == "stāvus")].shape[0], 5])

# stāvi VERB
print( "stāvi", "VERB",  df_non_nouns[(df_non_nouns['lemma'] == "stāvi")].shape[0],  0 )
temp_list.append(["stāvi", "VERB",  df_non_nouns[(df_non_nouns['lemma'] == "stāvi")].shape[0],  0])

# stāvēt VERB
stavet_WRONG_list = [
    146, 1247, 1263, 1743, 3103, 3158, 4604, 4726,
    5271, 5407, 5410, 6188, 6562, 6777, 10193
]
print( "stāvēt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "stāvēt")].shape[0],  15 )
temp_list.append(["stāvēt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "stāvēt")].shape[0],  15])

# Matis PROPN
6541
print( "Matis", "PROPN",  df_non_nouns[(df_non_nouns['lemma'] == "Matis")].shape[0], 1 )
temp_list.append(["Matis", "PROPN",  df_non_nouns[(df_non_nouns['lemma'] == "Matis")].shape[0], 1 ])

# galva PROPN
2394
print( "galva", "PROPN", df_non_nouns[(df_non_nouns['lemma'] == "galva")].shape[0],  1 )
temp_list.append(["galva", "PROPN", df_non_nouns[(df_non_nouns['lemma'] == "galva")].shape[0],  1])

# galviņu PROPN
1537
print( "galviņu", "PROPN", df_non_nouns[(df_non_nouns['lemma'] == "galviņu")].shape[0],  1 )
temp_list.append(["galviņu", "PROPN", df_non_nouns[(df_non_nouns['lemma'] == "galviņu")].shape[0],  1])

# celties VERB
print( "celties", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "celties")].shape[0],  0 )
temp_list.append(["celties", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "celties")].shape[0],  0])

# cirst VERB
print( "cirst", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "cirst")].shape[0],  0 )
temp_list.append(["cirst", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "cirst")].shape[0],  0])

# cirsties VERB
print( "cirsties", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "cirsties")].shape[0], 0 )
temp_list.append(["cirsties", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "cirsties")].shape[0], 0])

# mutīt VERB
mutit_WRONG_list = [
2591,
2596,
5036,
8080,
8267,
16659
]
print( "mutīt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "mutīt")].shape[0],  6 )
temp_list.append(["mutīt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "mutīt")].shape[0],  6])

# rakt VERB
6505
print( "rakt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "rakt")].shape[0],  1 )
temp_list.append(["rakt", "VERB", df_non_nouns[(df_non_nouns['lemma'] == "rakt")].shape[0],  1])

def write_missed_nouns_to_csv():
    with open("_51_missed_nouns.csv", "w") as out_f:
        writer = csv.writer(out_f)
        writer.writerow(["lemma", "vardskira", "TOTAL", "MISSED_NOUNS"])
        writer.writerows(temp_list)


stāvs ADJ 17 10
stāvi ADV 5 1
stāvus ADV 11 5
stāvi VERB 5 0
stāvēt VERB 316 15
Matis PROPN 1 1
galva PROPN 1 1
galviņu PROPN 1 1
celties VERB 4 0
cirst VERB 6 0
cirsties VERB 2 0
mutīt VERB 6 6
rakt VERB 1 1


In [12]:
# write_missed_nouns_to_csv()

In [13]:
# changing MISSED_NOUNS to correct lemmas and postags

# stāvs ADJ
stavs_WRONG_list = [
    1724, 7568, 7751, 12533, 14993, 15203, 15607, 15617, 15713, 15762, 3731, 3801, 3896, 
    5574, 15154, 15808, 146, 1247, 1263, 1743, 3103, 3158, 4604, 4726, 5271, 5407, 5410, 6188, 6562, 6777, 10193
]

# mutīt VERB
mutit_WRONG_list = [ 2591, 2596, 5036, 8080, 8267, 16659 ]

# Matis PROPN
matis_PROPN = [ 6541 ]

# galva PROPN
galva_PROPN = [ 2394 ]

# galviņu PROPN
galvinu_PROPN = [ 1537 ]

# rakt VERB
rakt_VERB = [ 6505 ]

for index, row in df.iterrows():
    if index in stavs_WRONG_list:
        df.loc[index, "lemma"] = "stāvs"
        df.loc[index, "vardskira"] = "NOUN"
    elif index in mutit_WRONG_list:
        df.loc[index, "lemma"] = "mutīte"
        df.loc[index, "vardskira"] = "NOUN"
    elif index in matis_PROPN:
        df.loc[index, "lemma"] = "mats"
        df.loc[index, "vardskira"] = "NOUN"
    elif index in galva_PROPN:
        df.loc[index, "lemma"] = "galva"
        df.loc[index, "vardskira"] = "NOUN"
    elif index in galvinu_PROPN:
        df.loc[index, "lemma"] = "galviņa"
        df.loc[index, "vardskira"] = "NOUN"
    elif index in rakt_VERB:
        df.loc[index, "lemma"] = "roka"
        df.loc[index, "vardskira"] = "NOUN"
    else:
        pass
    

In [14]:
def save_altered_non_nouns():
    list_of_lists = [ stavs_WRONG_list, mutit_WRONG_list, matis_PROPN, galva_PROPN, galvinu_PROPN, rakt_VERB ]
    list_of_indexes = [el for sub in list_of_lists for el in sub]

    df_non_nouns["atpazits"] = True
    for idx, row in df_non_nouns.iterrows():
        if idx in list_of_indexes:
            df_non_nouns.loc[idx, "atpazits"] = False
    
save_altered_non_nouns()


In [15]:
#     df_non_nouns.to_csv("_53_true_false_recognized_non_nouns.csv")


In [16]:
df_cels = df.loc[df['lemma'] == "ceļš"]
# df_cels.shape[0] # 814
# df_cels

# manuāli pārbaudīti un pierakstīti Indeksi, kur "ceļš" vietā vajadzēja būt atpazītam "celis"  # 129 no 814 tādi

kermenis_no_cels = [
    96, 912, 221, 746, 747, 758, 935, 936, 938, 939, 940, 941, 944, 950, 951, 954, 996,
    999, 1048, 1095, 1099, 1300, 1534, 2116, 2406, 2618,
    2657, 2661, 2671, 2675, 3155, 3553, 3879, 4171, 4194, 4203, 4479, 4578, 4709, 4710, 4711, 4735,
    5306, 5404, 5497, 5596, 5616, 5621, 5622, 5771, 6227, 6231, 6279, 6281, 6604, 6631, 6754, 6813,
    7186, 7235, 7274, 7303, 7682, 7712, 7723, 7824, 7851, 7988, 8050, 8434, 8440, 8500, 8512, 8710,
    8726, 8981, 9061, 9161, 9200, 9225, 9237, 9289, 9345, 9403, 9433, 9439, 9549, 9788, 9808, 10017, 10132,
    10134, 10378, 10404, 10428, 10473, 10736, 11162, 11412,
    11529, 11644, 12836, 13035, 13306, 13845, 13853, 13856,
    14031, 14224, 14226, 14789, 15554, 15915, 16015, 16094,
    16096, 16156, 16316, 17235, 17262, 17699, 17704, 17720,
    17839, 17984, 17985, 17995, 18181, 18662
]

# len(kermenis_no_cels) # 129
df_cels["atpazits"] = True

for index, row in df_cels.iterrows():
    if index in kermenis_no_cels:
        df_cels.loc[index, "atpazits"] = False       

df_cels = df_cels.sort_values(['atpazits'], ascending=[True])


def write_all_CELS_to_csv():
    df_cels.to_csv("_52_cels_atpazits_False_True.csv")

<ipython-input-16-73e9b3d4acd6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cels["atpazits"] = True
/Users/Haralds/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
# write_all_CELS_to_csv()

In [18]:
df_celis = df.loc[df['lemma'] == "celis"]
# df_celis.shape[0]  # 10
# tikai 10 formas "celi" un "celim". no tām 3 ir kļūdaini uzrakstīts "ceļš" -- "ceļi" vietā "celi" :
# 6860
# 15545
# 15759

In [19]:
df_celgals = df.loc[df['lemma'] == "ceļgals"]
# df_celgals.shape[0]  # 36
# vienā ir runa par lauku ceļa "ceļgalu"
# 16121    # kad izkāpu « Druvas » ceļgalā , mani apņēma lauku klusums

In [20]:
df_roka = df.loc[df['lemma'] == "roka"]
# df_roka.shape[0]  # 2874
# izskatīts līdz 1371 - apmēram desmitā daļa - visi pareizi atpazīti

In [21]:
df_roku = df.loc[df['fleksiva_forma'] == "roku"]
# df_roku.shape[0]  # 898
# izskatīts līdz 4007 - apmēram piektā daļa - visi pareizi atpazīti

In [22]:
df_cirta = df.loc[df['lemma'] == "cirta"]
# df_cirta.shape[0]  # 30
df_cirta

tris_nepareizas_cirtas = [ 481, 1044, 7889 ]

for index, row in df.iterrows():
    if index in tris_nepareizas_cirtas:
        df.loc[index, "vardskira"] = "not-NOUN"


In [23]:
# saglabājam sākotnējo dataframe iekš "df_initial" pirms sākt izmaiņas
df_initial = df.copy()

# izmest no dataframe visas rindas, kur "vardskira" nav "NOUN" – 

df = df.drop(df[df['vardskira'] != "NOUN"].index)

df.shape

(18521, 6)

In [24]:
# visus "ceļš", kas ir iekš manuāli pārbaudītā pareizā saraksta "kermenis_no_cels" pārnest uz "celis"

for index, row in df.iterrows():
    if row["lemma"] == "ceļš" and index in kermenis_no_cels:
        df.loc[index, "lemma"] = "celis"

# lemmas "celis" skaits palielinājies uz 138
# df_celis.shape[0] # 138

# izdzēst visus atlikušos nepareizos "ceļš"
df = df.drop(df[(df['lemma'] == "ceļš")].index)

df.shape

(17835, 6)

In [25]:
# pārbaudīti aizdomīgie un pārvērsti uz pareizām formām

aizdomigi = {
    "mutītis": "mutīte",
    "sejs": "seja",
    "roks": "roka",
    "krūšu": "krūts",
    "deguntiņa": "deguntiņš",
    "deguntiņs": "deguntiņš"
}

for index, row in df.iterrows():
    if row["lemma"] in aizdomigi.keys():
        df.loc[index, "lemma"] = aizdomigi[row["lemma"]]
#         print(row["lemma"], row["teksta_fragments"])


In [26]:
# TODO – šeit noglabāt pilnu atjaunot df uz csv failu .... 
df.shape

(17835, 6)

In [27]:
# GALA lielais fails, ar attīrītiem non-nouns, labotiem "ceļš" un citiem sīkumiem precizētiem
# df.to_csv("_54_PRECIZETS_kermena_vardi_vardskira_konteksts_17835_attiriti.csv")

In [28]:
# biežāk izplatītie ķermeņa vārdi LEMMAS formā
counts_df = df.value_counts(["lemma"])
# counts_df.head(74)
counts_df
# type(counts_df)

lemma     
roka          2878
acs           2659
galva         1509
seja          1335
kāja          1016
mats           686
plecs          544
lūpa           529
pirksts        472
mute           457
krūts          449
vaigs          417
mugura         369
kakls          339
piere          335
auss           273
elpa           267
miesa          263
stāvs          256
augums         240
zobs           185
deguns         167
klēpis         165
āda            145
plauksta       140
celis          138
elkonis        125
sāns           117
pēda           106
ķermenis       101
loceklis        91
galviņa         80
rociņa          78
uzacs           71
vēders          61
delna           55
sejiņa          51
zods            48
plaksts         45
paduse          43
actiņa          42
dvaša           40
pirkstiņš       40
kājiņa          38
papēdis         37
ceļgals         36
nāss            32
vaidziņš        31
plakstiņš       28
cirta           27
mutīte          25
gurns           24
p

In [29]:
# converting value_counts pandas series to dataframe
# lemmas_as_df = counts_df.to_frame()

lemmas_as_df = pd.DataFrame(counts_df)
lemmas_as_df = lemmas_as_df.reset_index()
lemmas_as_df.columns = ["Latvian", "counts"]
# lemmas_as_df = pd.DataFrame([counts_df.rename(None)]).T
# lemmas_as_df.columns = lemmas_as_df.columns.droplevel(0)
# lemmas_as_df = lemmas_as_df.reset_index(drop=True)

# lemmas_as_df["percents"] = lemmas_as_df["0"].value_counts(normalize=True) * 100
# df['gender'].value_counts(normalize=True) * 100

lemmas_as_df['percentage']= lemmas_as_df['counts']/(lemmas_as_df['counts'].sum()) * 100
lemmas_as_df['percentage'] = lemmas_as_df['percentage'].round(2)

lemmas_as_df

,Latvian,counts,percentage
0,roka,2878,16.14
1,acs,2659,14.91
2,galva,1509,8.46
3,seja,1335,7.49
4,kāja,1016,5.70
5,mats,686,3.85
6,plecs,544,3.05
7,lūpa,529,2.97
8,pirksts,472,2.65
9,mute,457,2.56


In [30]:
lemmas_as_df

citi_counts = 0
citi_percentages = 0

for index, row in lemmas_as_df.iterrows():
    if index > 9:
#         print(row["Latvian"])
        citi_counts += row["counts"]
        citi_percentages += row["percentage"]
        
print(citi_percentages)     
print(citi_counts)     

32.24999999999999
5750


In [31]:
sum_row = ["CITI VĀRDI", citi_counts, round(citi_percentages,2), "OTHER WORDS"]
sum_row



['CITI VĀRDI', 5750, 32.25, 'OTHER WORDS']

In [32]:
lat_to_eng = {
"roka": "arm",
"acs": "eye",
"galva": "head",
"seja": "face",
"kāja": "leg",
"mats": "hair",
"plecs": "shoulder",
"lūpa": "lip",
"pirksts": "finger",
"mute": "mouth"
}

eng_list = list(lat_to_eng.values())
eng_list



['arm',
 'eye',
 'head',
 'face',
 'leg',
 'hair',
 'shoulder',
 'lip',
 'finger',
 'mouth']

In [33]:
top_10_lemmas = lemmas_as_df.copy()

top_10_lemmas = top_10_lemmas.drop(top_10_lemmas.index[10:])
top_10_lemmas["English"] = eng_list  # adding English lemmas
top_10_lemmas.loc[10] = sum_row  # adding a row of sums for Other Words

top_10_lemmas.index = np.arange(1, len(top_10_lemmas) + 1)  # reindexing to start from 1

top_10_lemmas = top_10_lemmas[["English", "Latvian", "counts", "percentage"]]   # rearranging order of columns

top_10_lemmas

,English,Latvian,counts,percentage
1,arm,roka,2878,16.14
2,eye,acs,2659,14.91
3,head,galva,1509,8.46
4,face,seja,1335,7.49
5,leg,kāja,1016,5.70
6,hair,mats,686,3.85
7,shoulder,plecs,544,3.05
8,lip,lūpa,529,2.97
9,finger,pirksts,472,2.65
10,mouth,mute,457,2.56


In [35]:
top_10_lemmas.to_excel('_87_top10_lemmas_latviski_angliski_procenti.xlsx')

In [36]:
# lemmas_as_df.to_csv('_81_PRECIZETS_atslegvardi_lemmas_count.csv')

In [37]:
# TODO - turpināt šeit ar pareizas statistikas izveidi uz 2 csv failiem

In [38]:
# noglabājam uz failu:
# counts_df.to_csv('_55_PRECIZETS_atslegvardi_lemmas_count.csv')

In [39]:
# 10 biežāk izplatītie ķermeņa vārdi FLEKSĪVĀ formā – šis visdrīzāk nebūs nepieciešams, taču uztaisīju tādu failu.
counts_tokens = df.value_counts(["fleksiva_forma", "vardskira"])
# counts_tokens.head(10)

In [40]:
# noglabājam uz failu:
# counts_tokens.to_csv('_23_atslegvardi_fleksivas_formas_POS_count.csv')

In [41]:
df.head()

,fleksiva_forma,lemma,vardskira,indekss_teksta,teksta_fragments,faila_nosaukums
146,stāvu,stāvs,NOUN,9760,", tagad cieši apkampa viņas stāvu . Viņai pie šī apskāviena",1894AspazijaCinaParNakamibuNovele
1247,stāvu,stāvs,NOUN,1252,"mani un piekļāvās ar visu stāvu , tādu lokanu , kuru",1905BirzniekuSofijaIzMusuDienamStasts
1263,stāvu,stāvs,NOUN,1465,"raudzījās manī . Visu viņas stāvu , seju un saliektos pirkstus",1905BirzniekuSofijaIzMusuDienamStasts
1743,stāvu,stāvs,NOUN,666,vilnis tie nolija pār viņas stāvu … Noslīdēja no vienas rokas,1906ZemgaliesuBirutaVitaRozeSkice
3103,stāvu,stāvs,NOUN,890,"uz palmu birstalām , tavu stāvu es redzēšu starp cipresēm slaidu",1921ZamaicaLucijaLietusTelojums


In [42]:
# nokopējam df, lai turpmākas kalkulācijas ar df2 neietekmētu sākotnējo df
df2 = df.copy()

In [43]:
# faila apstrāde, lai pievienotu 4 ķermeņa vārdu tipus – 
atslegvardi_structured = pd.read_json('03_long_dict.json')

kermena_vardu_tips_obj = {}
for big_key, cluster in atslegvardi_structured.items():
    current_key = big_key
    current_cluster = []
    for wordlist in cluster:
        if type(wordlist) == list:
#             print(wordlist, type(wordlist))
            for el in wordlist:
                current_cluster.append(el)
    kermena_vardu_tips_obj[current_key] = current_cluster
# print(kermena_vardu_tips_obj)


In [44]:
# faila apstrāde, lai pievienotu 4 ķermeņa vārdu tipus – 

num_of_types = []

for index, row in df2.iterrows():
    for key, wordlist in kermena_vardu_tips_obj.items():
#         print(key, wordlist)
        if row["fleksiva_forma"] in wordlist:
            df2.at[index, "kermena_vardu_tips"] = key
#             row["kermena_vardu_tips"] = key
#             print(key, row["lemma"], row["kermena_vardu_tips"])
        if key not in num_of_types:
            num_of_types.append(key)

print(num_of_types)


['VISS_KERMENIS', 'KERMENA_DALAS', 'GALVA_UN_DALAS', 'CITI_VARDI']


In [45]:
df2.shape

(17835, 7)

In [46]:
# Dataframe papildināts ar kolonnu "kermena_vardu_tips"
df2.sort_values('faila_nosaukums')
df2.head(10)

,fleksiva_forma,lemma,vardskira,indekss_teksta,teksta_fragments,faila_nosaukums,kermena_vardu_tips
146,stāvu,stāvs,NOUN,9760,", tagad cieši apkampa viņas stāvu . Viņai pie šī apskāviena",1894AspazijaCinaParNakamibuNovele,VISS_KERMENIS
1247,stāvu,stāvs,NOUN,1252,"mani un piekļāvās ar visu stāvu , tādu lokanu , kuru",1905BirzniekuSofijaIzMusuDienamStasts,VISS_KERMENIS
1263,stāvu,stāvs,NOUN,1465,"raudzījās manī . Visu viņas stāvu , seju un saliektos pirkstus",1905BirzniekuSofijaIzMusuDienamStasts,VISS_KERMENIS
1743,stāvu,stāvs,NOUN,666,vilnis tie nolija pār viņas stāvu … Noslīdēja no vienas rokas,1906ZemgaliesuBirutaVitaRozeSkice,VISS_KERMENIS
3103,stāvu,stāvs,NOUN,890,"uz palmu birstalām , tavu stāvu es redzēšu starp cipresēm slaidu",1921ZamaicaLucijaLietusTelojums,VISS_KERMENIS
3158,stāvu,stāvs,NOUN,1358,"šīs krūtis , visu manu stāvu ? Kas būs šīs telpas",1921ZamaicaLucijaMurgiStasts,VISS_KERMENIS
4604,stāvu,stāvs,NOUN,1908,"ap kājām un ap visu stāvu . Viņa likās peldus ,",1927GailiteAngelikaCorArdensStasts,VISS_KERMENIS
4726,stāvam,stāvs,NOUN,2635,uzmeta pētošu skatu viņas vārīgam stāvam un ar īstu mincheniešu jautrību,1927GailiteAngelikaKazuCelojumsStasts,VISS_KERMENIS
5271,stāvu,stāvs,NOUN,3150,"rokās šo tumšo , mulsinošo stāvu . „ Izolde !.. Izolde",1928BendrupeMirdzaJaunatneStasts,VISS_KERMENIS
5407,stāvu,stāvs,NOUN,9248,"Ivetu stāvēja Izolde . Viņas stāvu apņēma melns samta uzpletnis ,",1928BendrupeMirdzaJaunatneStasts,VISS_KERMENIS


In [47]:
# izveidojam atsevišķu df priekš "ķermeņa vārdu tipu" grupēšanas
kermena_vardu_tipi_df = df2[["faila_nosaukums", "kermena_vardu_tips"]].copy()
kermena_vardu_tipi_df.head()

,faila_nosaukums,kermena_vardu_tips
146,1894AspazijaCinaParNakamibuNovele,VISS_KERMENIS
1247,1905BirzniekuSofijaIzMusuDienamStasts,VISS_KERMENIS
1263,1905BirzniekuSofijaIzMusuDienamStasts,VISS_KERMENIS
1743,1906ZemgaliesuBirutaVitaRozeSkice,VISS_KERMENIS
3103,1921ZamaicaLucijaLietusTelojums,VISS_KERMENIS


In [48]:
# saskaitām cik katrā tekstā ir no katra "ķermeņu vārdu tipa"
kermena_vardu_tipi_df = kermena_vardu_tipi_df.value_counts(["faila_nosaukums", "kermena_vardu_tips"]).reset_index(name='counts')
kermena_vardu_tipi_df.head(10)

,faila_nosaukums,kermena_vardu_tips,counts
0,1964BrodeleAnnaZilaisZvirbulisStasts,GALVA_UN_DALAS,354
1,1912AntijaAinaAreStasts,GALVA_UN_DALAS,254
2,1964BrodeleAnnaZilaisZvirbulisStasts,KERMENA_DALAS,246
3,1983SvireMaraArVakardienuKabataStasts,KERMENA_DALAS,232
4,1949BrodeleAnnaMartasDienasgramataStasts,GALVA_UN_DALAS,219
5,1935TirzmalieteSievieteStasts,KERMENA_DALAS,208
6,1935TirzmalieteSievieteStasts,GALVA_UN_DALAS,196
7,1949BrodeleAnnaMartasDienasgramataStasts,KERMENA_DALAS,195
8,1972AvotinaDainaZuzeStasts,KERMENA_DALAS,185
9,1983SvireMaraArVakardienuKabataStasts,GALVA_UN_DALAS,181


In [49]:
# dataframe apstrāde, lai iegūtu visus 4 "kermena_vardu_tipus" vienā tabulā 

temp_obj = {}

for index, row in kermena_vardu_tipi_df.iterrows():
    if row["faila_nosaukums"] not in temp_obj:
        temp_obj[row["faila_nosaukums"]] = {}
        temp_obj[row["faila_nosaukums"]]["VISS_KERMENIS"] = 0
        temp_obj[row["faila_nosaukums"]]["KERMENA_DALAS"] = 0
        temp_obj[row["faila_nosaukums"]]["GALVA_UN_DALAS"] = 0
        temp_obj[row["faila_nosaukums"]]["CITI_VARDI"] = 0    
    temp_obj[row["faila_nosaukums"]][row["kermena_vardu_tips"]] = row["counts"]
        
# print(temp_obj)      
# kermena_vardu_tipi_df
# type(temp_obj)
# len(temp_obj)

temp_list = []
for key, val in temp_obj.items():
    temp_list.append([key, val['VISS_KERMENIS'] or 0, val['KERMENA_DALAS'] or 0, val['GALVA_UN_DALAS'] or 0, val['CITI_VARDI'] or 0 ])

temp_list.sort()
# print(temp_list)


In [50]:
# hronoloģiski sakārtoti visi teksti un kermena vārdu skaits katrā no 4 tipiem —–
headers = [ 'faila_nosaukums', 'VISS_KERMENIS', 'KERMENA_DALAS', 'GALVA_UN_DALAS', 'CITI_VARDI']
dfd = pd.DataFrame(temp_list, columns=headers)

# pievienojam arī saskaitītu KOPĒJO ķermeņa vārdu skaitu katram tekstam ——
dfd["TOTAL"] = dfd['VISS_KERMENIS'] + dfd['KERMENA_DALAS'] + dfd['GALVA_UN_DALAS'] + dfd['CITI_VARDI']
dfd.head()

,faila_nosaukums,VISS_KERMENIS,KERMENA_DALAS,GALVA_UN_DALAS,CITI_VARDI,TOTAL
0,1893MedinskaMarijaSērdienīteStasts,7,15,13,3,38
1,1894AspazijaCinaParNakamibuNovele,14,70,76,8,168
2,1895ZaliteHermineEjUnIzglitojiesStasts,1,7,8,0,16
3,1896ZaliteHermineIzlīdzētsStasts,0,6,6,1,13
4,1896ZaliteHermineMūsuCeliSkirasStasts,1,12,26,1,40


In [51]:
# no izejas failiem iegūstam "tokens_count" katrā teksta failā.  "tokens_count" = vārdi + pieturzīmes.

rootdir = '/Users/Haralds/Desktop/H/DH/LFMI/zitas-projekts/lnb_preprocessing/csv'

list_of_filelengths = []
for subdir, dirs, files in os.walk(rootdir):
    for filename in sorted(files):
        with open(os.path.join(rootdir, filename)) as f:      
#             print(filename[:-4], len(f.readlines()) - 1 )
            list_of_filelengths.append([filename[:-4], len(f.readlines()) - 1 ])

# print(list_of_filelengths)

fl_df = pd.DataFrame(list_of_filelengths, columns=('faila_nosaukums', 'tokens_count'))
fl_df.head()

,faila_nosaukums,tokens_count
0,1893MedinskaMarijaSērdienīteStasts,7148
1,1894AspazijaCinaParNakamibuNovele,17619
2,1895ZaliteHermineEjUnIzglitojiesStasts,2374
3,1896ZaliteHermineIzlīdzētsStasts,1295
4,1896ZaliteHermineMūsuCeliSkirasStasts,5073


In [52]:
# pievienojam pie dataframe "tokens_count" un "kv_procentos", kas parāda, cik procenti no "tokens_count" 
# ir kermena vārdi –– 

combined_df = pd.merge(dfd, fl_df, how="outer", on=["faila_nosaukums"])
# combined_df.head()

combined_df["kv_procentos"] = ( combined_df["TOTAL"] / combined_df["tokens_count"] ) * 100
combined_df["kv_procentos"] = combined_df["kv_procentos"].round(2)
combined_df

,faila_nosaukums,VISS_KERMENIS,KERMENA_DALAS,GALVA_UN_DALAS,CITI_VARDI,TOTAL,tokens_count,kv_procentos
0,1893MedinskaMarijaSērdienīteStasts,7,15,13,3,38,7148,0.53
1,1894AspazijaCinaParNakamibuNovele,14,70,76,8,168,17619,0.95
2,1895ZaliteHermineEjUnIzglitojiesStasts,1,7,8,0,16,2374,0.67
3,1896ZaliteHermineIzlīdzētsStasts,0,6,6,1,13,1295,1.00
4,1896ZaliteHermineMūsuCeliSkirasStasts,1,12,26,1,40,5073,0.79
5,1896ZaliteHermineNoTumsībasPieGaismasStasts,2,9,16,1,28,4771,0.59
6,1898BirzniekuLatinaLaimigaisTevsStasts,4,16,12,0,32,3102,1.03
7,1899BirzniekuLatinaKarsuPartijaStasts,3,17,34,1,55,4906,1.12
8,1899BirzniekuLatinaPusratiStasts,2,13,15,1,31,3104,1.00
9,1899BrigadereAnnaPatversmeStasts,2,48,51,7,108,9324,1.16


In [59]:
# noglabājam kermena_vardi_absolute_counts uz csv failu:
# combined_df.to_csv('_56_PRECIZETS_kermena_vardi_absolute_counts.csv')
# combined_df.to_excel('_88_kermena_vardi_4_grupas_absolute_counts.xlsx')

In [54]:
# pārvēršam absolūtos skaitļus uz relatīvām vērtībām – lai redzētu, kuros tekstos ir lielāks
# ķermeņa vārdu ĪPATSVARS

kermena_vardi_relative_counts_df = combined_df.copy()

kermena_vardi_relative_counts_df["VISS_KERMENIS"] = (( kermena_vardi_relative_counts_df["VISS_KERMENIS"] / kermena_vardi_relative_counts_df["tokens_count"] ) * 100 ).round(4)
kermena_vardi_relative_counts_df["KERMENA_DALAS"] = (( kermena_vardi_relative_counts_df["KERMENA_DALAS"] / kermena_vardi_relative_counts_df["tokens_count"] ) * 100 ).round(4)
kermena_vardi_relative_counts_df["GALVA_UN_DALAS"] = (( kermena_vardi_relative_counts_df["GALVA_UN_DALAS"] / kermena_vardi_relative_counts_df["tokens_count"] ) * 100 ).round(4)
kermena_vardi_relative_counts_df["CITI_VARDI"] = (( kermena_vardi_relative_counts_df["CITI_VARDI"] / kermena_vardi_relative_counts_df["tokens_count"] ) * 100 ).round(4)

# sakārtojam tekstus, sākot ar VISLIELĀKO ķermeņa vārdu īpatsvaru
kermena_vardi_relative_counts_df = kermena_vardi_relative_counts_df.sort_values(['kv_procentos'], ascending=[False])

kermena_vardi_relative_counts_df.head(10)

,faila_nosaukums,VISS_KERMENIS,KERMENA_DALAS,GALVA_UN_DALAS,CITI_VARDI,TOTAL,tokens_count,kv_procentos
29,1905ZemgaliesuBirutaJurmalasBernsSkice,0.4125,0.6601,2.6403,0.4125,50,1212,4.13
251,1994ZemdegaAinaAtgriesanasStasts,0.1399,1.7716,1.6783,0.3263,84,2145,3.92
119,1930BangaTijaAktriseSkice,0.3317,0.9950,2.3217,0.1658,23,603,3.81
255,1998AbeleIngaNatresSkice,0.0990,1.5842,1.6832,0.3960,38,1010,3.76
46,1911LicisuPaulaDivasRokasSkice,0.0000,2.8476,0.6700,0.1675,22,597,3.69
59,1921ZamaicaLucijaMurgiStasts,0.5533,0.9682,1.7289,0.3804,105,2892,3.63
206,1973BlumfeldeIreneDejotajaTelojums,0.5046,1.0092,1.8807,0.1835,78,2180,3.58
58,1921ZamaicaLucijaLietusTelojums,0.4795,1.3014,1.5068,0.2055,51,1460,3.49
111,1929BendrupeMirdzaMarijaMadaļaStasts,0.2913,1.3506,1.5890,0.2119,130,3776,3.44
248,1994IkstenaGadijumsArKornelijuStasts,0.0000,1.2458,1.4950,0.5814,40,1204,3.32


In [60]:
# noglabājam kermena_vardi_RELATIVE_counts uz csv failu:
# kermena_vardi_relative_counts_df.to_csv('_57_PRECIZETS_kermena_vardi_RELATIVE_counts.csv')
kermena_vardi_relative_counts_df.to_excel('_89_kermena_vardi_4_grupas_RELATIVE_counts.xlsx')

In [56]:
avges_list = kermena_vardi_relative_counts_df["kv_procentos"].tolist()
avges_list

len_of_list = len(avges_list)
len_of_list

sum_of_list = sum(avges_list)
avg_value = sum_of_list / len_of_list
avg_value  # per 100 words in aggregate corpus of 259 texts

1.6281853281853285

In [57]:


# variance = squared deviation
varnce_list = []

for item in avges_list:
    varnce_list.append((item - avg_value) ** 2)
#     print((item - avg_value) ** 2)
 
# print(varnce_list)

real_variance = (sum(varnce_list))/len_of_list
# real_variance

calculated_std_dev = math.sqrt(real_variance)
calculated_std_dev

0.7645980777373946

In [58]:
# calculate stdandard deviation for average body word counts in texts

std_dev = statistics.stdev(kermena_vardi_relative_counts_df["kv_procentos"])
std_dev

0.7660784239011365